# Introduction

I will explore the problem in following stages:

1.  **Hypothesis Generation** – understanding the problem better by brainstorming possible factors that can impact the outcome
2.  **Data Exploration** – looking at categorical and continuous feature summaries and making inferences about the data.
3.  **Data Cleaning** – imputing missing values in the data and checking for outliers
4.  **Feature Engineering** – modifying existing variables and creating new ones for analysis
5.  **Model Building** – making predictive models on the data


## 1. Hypothesis Generation

This involves understanding the problem and making some hypothesis about what could potentially have a good impact on the outcome. This is done BEFORE looking at the data, and we end up creating a laundry list of the different analysis which we can potentially perform if data is available.

### The Problem Statement

Understanding the problem statement is the first and foremost step:

> In this competition, you will forecast the demand of a product for a given week, at a particular store. The dataset you are given consists of 9 weeks of sales transactions in Mexico. Every week, there are delivery trucks that deliver products to the vendors. Each transaction consists of sales and returns. Returns are the products that are unsold and expired. The demand for a product in a certain week is defined as the sales this week subtracted by the return next week.

So the idea is to find out the demand of a product (sales - returns) per client, and store which impacts the sales of a product. Let’s think about some of the analysis that can be done and come up with certain hypothesis.

### The Hypotheses

I came up with the following hypothesis while thinking about the problem. Since we’re talking about stores and products, lets make different sets for each.

**Store/Client Level Hypotheses:**

1.  **Town type:** Stores located in urban or Tier 1 towns should have higher sales because of the higher income levels of people there.
2.  **Population Density:** Stores located in densely populated areas should have higher sales because of more demand.
3.  **Store Capacity:** Stores which are very big in size should have higher sales as they act like one-stop-shops and people would prefer getting everything from one place
4.  **Competitors:** Stores having similar establishments nearby should have less sales because of more competition.
5.  **Marketing:** Stores which have a good marketing division should have higher sales as it will be able to attract customers through the right offers and advertising.
6.  **Location:** Stores located within popular marketplaces should have higher sales because of better access to customers.
7.  **Customer Behavior:** Stores keeping the right set of products to meet the local needs of customers will have higher sales.
8.  **Ambiance:** Stores which are well-maintained and managed by polite and humble people are expected to have higher footfall and thus higher sales.
9.  **Season:** Store should sell more after customer's pay day: after 15th or 30th of the month

**Product Level Hypotheses:**

1.  **Brand:** Branded products should have higher sales because of higher trust in the customer.
2.  **Packaging:** Products with good packaging can attract customers and sell more.
3.  **Utility:** Daily use products should have a higher tendency to sell as compared to the specific use products.
4.  **Display Area:** Products which are given bigger shelves in the store are likely to catch attention first and sell more.
5.  **Visibility in Store:** The location of product in a store will impact sales. Ones which are right at entrance will catch the eye of customer first rather than the ones in back.
6.  **Advertising:** Better advertising of products in the store will should higher sales in most cases.
7.  **Promotional Offers:** Products accompanied with attractive offers and discounts will sell more.


Lets move on to the data exploration where we will have a look at the data in detail.

## 2\. Data Exploration

I’ll be performing some basic data exploration here and come up with some inferences about the data.

The first step is to look at the data and try to identify the information which we hypothesized vs the available data. A comparison between the data dictionary on the competition page and out hypotheses is shown below:

![Image of Variables vs Hypothesis](files/../input-data/Variables_vs_Hyphotesis.png)

We can summarize the findings as:

** 9 Features Hypothesized but not found in actual data. **

** 5 Features Hypothesized as well as present in the data **

** 3 Features present in the data but not hypothesized. **


We find features which we hypothesized, but data doesn’t carry and vice versa. We should look for open source data to fill the gaps if possible. Let’s start by loading the required libraries and data. 

In [1]:
import pandas as pd
import numpy as np
import time
import csv

_start_time = time.time()

# define a easy timing function to use going forward
def tic():
    global _start_time 
    _start_time = time.time()

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('Time passed: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))
    
# utility function- display large dataframes in an html iframe
def df_display(df, lines=500):
    txt = ("<iframe " +
           "srcdoc='" + df.head(lines).to_html() + "' " +
           "width=1000 height=500>" +
           "</iframe>")

    return IPython.display.HTML(txt)


In [2]:
#Read files:
tic()
train = pd.read_csv('input-data/train.csv',
                           dtype  = {'Semana': 'int8',
                                     'Producto_ID':'int32',
                                     'Cliente_ID':'int32',
                                     'Agencia_ID':'uint16',
                                     'Canal_ID':'int8',
                                     'Ruta_SAK':'int32',
                                     'Venta_hoy':'float32',
                                     'Venta_uni_hoy': 'int8',
                                     'Dev_uni_proxima':'int8',
                                     'Dev_proxima':'float32',
                                     'Demanda_uni_equil':'int32'})
test = pd.read_csv('input-data/test.csv',
                           dtype  = {'Semana': 'int8',
                                     'Producto_ID':'int32',
                                     'Cliente_ID':'int32',
                                     'Agencia_ID':'uint16',
                                     'Canal_ID':'int8',
                                     'Ruta_SAK':'int32'})
tac()

Time passed: 0hour:0min:53sec


In [3]:
train.head()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,3


In [4]:
test.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277


In [5]:
# remove unnecessary fields in training data
train.drop(['Venta_uni_hoy', 'Venta_hoy','Dev_uni_proxima', 'Dev_proxima'], axis=1, inplace=True)

In [6]:
#Since test dataframe is not the same as train dataframe, we make them equal by removing and adding columns
train.insert(0, 'id', np.nan)
test.insert(7, 'Demanda_uni_equil', np.nan)

Now, probably one of the most important steps is to split the train set in train and validation. And never touch the validation set for any feature engineering, and to treat it just like the unseen test set

In [7]:
use_validation=True
if (use_validation):
    tic()
    val = train[train.Semana > 8] # Weeks 9
    train = train[train.Semana <=8] # Weeks 7,8
    tac()
    

Time passed: 0hour:0min:3sec


It is a good idea to combine both train and test data sets into one, perform feature engineering and then divide them later again. This saves the trouble of performing the same steps twice on test and train. Lets combine them into a dataframe ‘data’ with a ‘source’ column specifying where each observation belongs.

In [8]:
tic()
train['source']='train'
if (use_validation): val['source']='val'
test['source']='test'

if (use_validation): 
    data = pd.concat([train,val,test],ignore_index=True)
    print (train.shape, val.shape, test.shape, data.shape)
else:
    data = pd.concat([train,test],ignore_index=True)
    print (train.shape, test.shape, data.shape)
tac()


(63771751, 9) (10408713, 9) (6999251, 9) (81179715, 9)
Time passed: 0hour:0min:9sec


In [9]:
#We must now remove the target column=Demanda_uni_equil from the val dataset and keep it away
data['ix'] = data.index # assign and id column to later at the end map the val_targets back to the dataframe
if (use_validation):
    val_target = data[['ix','Demanda_uni_equil']].loc[data['source']=='val'] #save the val targets on a different dataframe
    data.loc[data['source']=='val','Demanda_uni_equil'] =  np.nan # replaces val targets with nan for feature engineering

Thus we can see that data has same #columns but rows equivalent to both test and train. Lets start by checking which columns contain missing values. (takes aprox 30 mins to run!)

In [ ]:
data.apply(lambda x: sum(x.isnull()))

There doesn't seem to be any missing values (other than the NaN we set on the test and train sets).

Lets look at some basic statistics for numerical variables.

In [10]:
data.describe()

/anaconda/envs/py35/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,ix
count,6.999251e+06,8.117972e+07,8.117972e+07,8.117972e+07,8.117972e+07,8.117972e+07,8.117972e+07,6.377175e+07,8.117972e+07
mean,3.499625e+06,6.341839e+00,2.533746e+03,1.384793e+00,2.116852e+03,1.803586e+06,2.095482e+04,7.226821e+00,4.058986e+07
std,2.020510e+06,2.313453e+00,4.069579e+03,1.467666e+00,1.488853e+03,2.406086e+06,1.867056e+04,2.174982e+01,2.343457e+07
min,0.000000e+00,3.000000e+00,1.110000e+03,1.000000e+00,1.000000e+00,2.600000e+01,4.100000e+01,0.000000e+00,0.000000e+00
25%,NaN,4.000000e+00,1.311000e+03,1.000000e+00,1.161000e+03,3.567150e+05,1.242000e+03,NaN,2.029493e+07
50%,NaN,6.000000e+00,1.613000e+03,1.000000e+00,1.286000e+03,1.193935e+06,3.054900e+04,NaN,4.058986e+07
75%,NaN,8.000000e+00,2.036000e+03,1.000000e+00,2.803000e+03,2.372538e+06,3.756900e+04,NaN,6.088479e+07
max,6.999250e+06,1.100000e+01,2.575900e+04,1.100000e+01,9.991000e+03,2.015152e+09,4.999700e+04,5.000000e+03,8.117971e+07


Some observations:

   Looking at Demanda_uni_equil (our target), or the amount of product sold per week, we find interesting things:
   
   **1)** The average is 7.22, so in average there is 7 units per week per store sold.
   
   **2)** Looking at the max of 5000, it looks very far fro the mean (3 orders of magnitude), so we must check for an outlier here or a store that is crazy different from the rest.
   
   **3)** Same behaviour we find on Dev_uni_proxima, Venta_hoy and Venta_uni_hoy
   
Looking at the nice data analysis made in R by Faviens, here: https://www.kaggle.com/fabienvs/grupo-bimbo-inventory-demand/notebook-8a62eda039a3b0b944cf/notebook we corroborate the outlier(s):
There is a massive client: Puebla Remision
   
![Image of size of Customers]( https://www.kaggle.io/svf/267812/783a24d1dd546819a44914f996b249e8/__results___files/figure-html/unnamed-chunk-16-1.png)
   

Moving to nominal (categorical) variable, lets have a look at the number of unique values in each of them.

In [45]:
data.apply(lambda x: len(x.unique()))

id                    6999252
Semana                      9
Agencia_ID                552
Canal_ID                    9
Ruta_SAK                 3620
Cliente_ID             890267
Producto_ID              1833
Demanda_uni_equil        2000
source                      3
ix                   81179715
dtype: int64

So, in train and test sets, we have 552 Agencies(depots), 890k clients (we might have some repeated clients due to typos when enterind data), 1833 products (we might have some repeated products here based on typos) and 3620 routes

In [46]:
# Let's see how many records we have per week
for i in range(3,12):
    print("Semana"+repr(i)+" =\t" + repr(data[data["Semana"]==i].Semana.count()))

Semana3 =	11165207
Semana4 =	11009593
Semana5 =	10615397
Semana6 =	10191837
Semana7 =	10382849
Semana8 =	10406868
Semana9 =	10408713
Semana10 =	3538385
Semana11 =	3460866


As stated in the Kaggle competition - Week 10 and 11 is sampled down aprox 70%. According to Kaggle, this was done so the scoring of candidates didn't take extremely long.

## 3\. Data Cleaning

This step involves imputing missing values and treating outliers. As we saw before, there are no missing values. Regarding outliers, there seem to be an obvious one, but we are going to see later on if its necessary to treat it differently.

My initial reaction would be to see if anything with the word REMISION is on the test set. if not, then delete it. See this discussion: https://www.kaggle.com/c/grupo-bimbo-inventory-demand/forums/t/22037/puebla-remission/126053

In [ ]:
#Let's find out who are the clients with the word REMISION on it
client_name = pd.read_csv('files/../input-data/cliente_tabla.csv')
cliente_id_name_train = pd.merge(train,client_name, on='Cliente_ID')
cliente_id_name_test = pd.merge(test,client_name, on='Cliente_ID')

In [ ]:
cliente_id_name_train.head()

In [ ]:
cliente_id_name_train[cliente_id_name_train.NombreCliente.str.contains('REMISION')].count()

As we can see above, the word "REMISION" shows up 140k times on the train set. Let's see the test set:

In [ ]:
cliente_id_name_test[cliente_id_name_test.NombreCliente.str.contains('REMISION')].count()

12k rows shows up the word REMISION on the test set. This implies that it has to be predicted as well. We cannot eliminate it.

## 4\. Feature Engineering

We explored some nuances in the data in the data exploration section. Lets move on to resolving them and making our data ready for analysis. We will also create some new variables using the existing ones in this section.

In [11]:
#First thing we need to do is to transform our target ( Demanda_uni_equil) to log(1 + demand) - this makes sense since we're 
#trying to minimize rmsle vs the mean which minimizes rmse. At the end of the modeling (for submission) we need to reverse it 
#by applying expm1(x)

data['log_target'] = np.log1p(data["Demanda_uni_equil"])

In [12]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,log_target
0,NaN,3,1110,7,3301,15766,1212,3.0,train,0,1.386294
1,NaN,3,1110,7,3301,15766,1216,4.0,train,1,1.609438
2,NaN,3,1110,7,3301,15766,1238,4.0,train,2,1.609438
3,NaN,3,1110,7,3301,15766,1240,4.0,train,3,1.609438
4,NaN,3,1110,7,3301,15766,1242,3.0,train,4,1.386294


In [13]:
#Let's also create all the grouping dataframes we are going to need 
tic()

global_mean = data['log_target'].mean()
prod_mean = data.groupby('Producto_ID').agg({'log_target': 'mean' })
client_mean = data.groupby('Cliente_ID').agg({'log_target': 'mean' })
prod_client_mean = data.groupby(['Producto_ID', 'Cliente_ID']).agg({'log_target': 'mean' })
semana_client_prod_mean = data.groupby(['Semana','Cliente_ID','Producto_ID']).agg({'log_target': 'mean'})
ruta_cliente_prod_mean = data.groupby(['Ruta_SAK','Cliente_ID','Producto_ID']).agg({'log_target': 'mean'})

tac()

Time passed: 0hour:2min:10sec


### Feature 1: Lags - Demand per client-product pair for prior weeks
Based on this blog: http://blog.nycdatascience.com/student-works/predicting-demand-historical-sales-data-grupo-bimbo-kaggle-competition/

As this script said: https://www.kaggle.com/bpavlyshenko/grupo-bimbo-inventory-demand/bimbo-xgboost-r-script-lb-0-457/code
It is important to know what were the previous weeks sales. If the previous week, too many products were supplied and they were not sold, the next week this product amount, supplied to the same store, will be decreased. So it is very important to included lag values of target variable as a feature to predict the next sales.

In [52]:
df = semana_client_prod_mean.reset_index() # we convert the index to columns for later use

In [53]:
df.head()

,Semana,Cliente_ID,Producto_ID,log_target
0,3,26,1182,3.688879
1,3,26,4767,3.761200
2,3,26,31393,3.044522
3,3,26,31690,3.761200
4,3,26,32962,1.386294


In [54]:
# Let's see how many records we have per week on the semana_cliente_Producto groups vs the raw dataset
for i in range(3,12):
    print("Semana"+repr(i)+" =\t" + repr(data[data["Semana"]==i].Semana.count())+ " (raw)\t" +
            repr(df[df["Semana"]==i].Semana.count()) + " (group)\t" +  
            repr(data[data["Semana"]==i].Semana.count() - df[df["Semana"]==i].Semana.count()) + " (diff)"
         )

Semana3 =	11165207 (raw)	11140707 (group)	24500 (diff)
Semana4 =	11009593 (raw)	10985436 (group)	24157 (diff)
Semana5 =	10615397 (raw)	10591171 (group)	24226 (diff)
Semana6 =	10191837 (raw)	10167946 (group)	23891 (diff)
Semana7 =	10382849 (raw)	10359111 (group)	23738 (diff)
Semana8 =	10406868 (raw)	10383302 (group)	23566 (diff)
Semana9 =	10408713 (raw)	10385350 (group)	23363 (diff)
Semana10 =	3538385 (raw)	3531921 (group)	6464 (diff)
Semana11 =	3460866 (raw)	3454394 (group)	6472 (diff)


As we can see above, there are repeated combinations of client-product on each week.

In [55]:
#Before we start adding lags and removing rows, let's see the size of our dataset
size_data = data.memory_usage().sum()
print(size_data)

4546064040


In [56]:
#here we add the number of lags we want
tic()
lag=4

for i in range(1,lag+1):
    df['Semana'] += 1
    df.rename(columns={df.columns[3]: 'Log_Target_mean_lag%d' %(i)}, inplace=True)
    data = pd.merge(data,df, how = 'left', on = ['Semana','Cliente_ID','Producto_ID']) #here we add the lag to the dataset
    data['Log_Target_mean_lag%d' %(i)].fillna(0, inplace=True) # we replace the client-product log mean NaN/Not found on the week before with ZERO
    data = data[data.Semana != i+2] # here we delete the week rows we dont have lags for
   
tac()

Time passed: 0hour:8min:24sec


In [57]:
#Let's see how many NaN or Nulls we have
data.apply(lambda x: sum(x.isnull()))

id                      31198430
Semana                         0
Agencia_ID                     0
Canal_ID                       0
Ruta_SAK                       0
Cliente_ID                     0
Producto_ID                    0
Demanda_uni_equil       17407964
source                         0
ix                             0
log_target              17407964
Log_Target_mean_lag1           0
Log_Target_mean_lag2           0
Log_Target_mean_lag3           0
Log_Target_mean_lag4           0
dtype: int64

The above looks correct! the only NaN shown are the variables that are not avaibable on the test set, everyting else looks good


In [58]:
# Let's see how many records we have per week and make sure we didn't delete any data from our important weeks
for i in range(3,12):
    print("Semana"+repr(i)+" =\t" + repr(data[data["Semana"]==i].Semana.count()))

Semana3 =	0
Semana4 =	0
Semana5 =	0
Semana6 =	0
Semana7 =	10382849
Semana8 =	10406868
Semana9 =	10408713
Semana10 =	3538385
Semana11 =	3460866


The above looks correct! we deleted the week rows we don't have lags for, and we kept the original amount of rows for the remaining weeks.

In [59]:
#Now let's see how much was the data set size reduced/increased
new_size_data = data.memory_usage().sum()
print("Dataset size changed in " + repr((new_size_data - size_data )*100/new_size_data) + "%")

Dataset size changed in -35.24333751141499%


Good! the data set was reduced in 27% (this means faster modeling)

In [60]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,log_target,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4
10191837,NaN,7,1110,7,3301,15766,1216,2,train,42982034,1.098612,0.693147,1.386294,1.098612,1.609438
10191838,NaN,7,1110,7,3301,15766,1238,2,train,42982035,1.098612,1.098612,0.693147,1.386294,1.609438
10191839,NaN,7,1110,7,3301,15766,1240,8,train,42982036,2.197225,0.000000,0.000000,2.079442,1.609438
10191840,NaN,7,1110,7,3301,15766,1242,2,train,42982037,1.098612,1.386294,1.098612,1.386294,1.386294
10191841,NaN,7,1110,7,3301,15766,1250,14,train,42982038,2.708050,0.693147,2.197225,1.386294,1.791759


### Feature 2: Rate of change of lags
It is also important to know what is the rate of change in demand from week to week in respect to the product-client mean

In [61]:
df = prod_client_mean.reset_index() # we convert the index to columns for later use
df.rename(columns={'log_target': 'PC_log_target_mean'}, inplace=True)

In [62]:
data = pd.merge(data,df, how = 'left', on = ['Producto_ID','Cliente_ID'])

In [63]:
data.tail()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,log_target,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,PC_log_target_mean
38197676,6999246,11,2057,1,1153,4379638,1232,NaN,test,81179710,NaN,0,0.000000,0.000000,1.386294,1.242453
38197677,6999247,10,1334,1,2008,970421,43069,NaN,test,81179711,NaN,0,1.386294,0.693147,0.693147,1.039721
38197678,6999248,11,1622,1,2869,192749,30532,NaN,test,81179712,NaN,0,0.000000,1.386294,1.098612,1.370160
38197679,6999249,11,1636,1,4401,286071,35107,NaN,test,81179713,NaN,0,0.000000,0.000000,0.000000,1.098612
38197680,6999250,11,1625,1,1259,978760,1232,NaN,test,81179714,NaN,0,0.000000,0.000000,0.000000,0.346574


In [64]:
tic()
for i in range(2,lag+1):
        data['rate_change%d' %(i)] = (data['Log_Target_mean_lag%d' %(i-1)] - data['Log_Target_mean_lag%d' %(i)])/data['PC_log_target_mean']
        data['rate_change%d' %(i)].fillna(0, inplace=True) # we replace NaN/Not found with ZERO
    
tac()

Time passed: 0hour:0min:2sec


In [65]:
#We don't need Qty_Ruta_Sak anymore
data.drop(['PC_log_target_mean'],axis=1,inplace=True)

In [66]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,log_target,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,rate_change2,rate_change3,rate_change4
0,NaN,7,1110,7,3301,15766,1216,2,train,42982034,1.098612,0.693147,1.386294,1.098612,1.609438,-0.541672,0.224814,-0.399194
1,NaN,7,1110,7,3301,15766,1238,2,train,42982035,1.098612,1.098612,0.693147,1.386294,1.609438,0.334524,-0.571872,-0.184102
2,NaN,7,1110,7,3301,15766,1240,8,train,42982036,2.197225,0.000000,0.000000,2.079442,1.609438,0.000000,-1.190852,0.269161
3,NaN,7,1110,7,3301,15766,1242,2,train,42982037,1.098612,1.386294,1.098612,1.386294,1.386294,0.244862,-0.244862,0.000000
4,NaN,7,1110,7,3301,15766,1250,14,train,42982038,2.708050,0.693147,2.197225,1.386294,1.791759,-0.822372,0.443386,-0.221693


###  Feature 3:  Calculates de sum of prior weeks Log mean Demands

In [67]:
#We want to sum the lags up until week 9, this means that we need to sum lag2 and up.
data['Lags_sum'] = 0
for i in range(1,lag+1):
    data['Lags_sum'] += data['Log_Target_mean_lag%d' %(i)]

In [68]:
data.tail()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,log_target,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,rate_change2,rate_change3,rate_change4,Lags_sum
38197676,6999246,11,2057,1,1153,4379638,1232,NaN,test,81179710,NaN,0,0.000000,0.000000,1.386294,0.000000,0.000000,-1.115772,1.386294
38197677,6999247,10,1334,1,2008,970421,43069,NaN,test,81179711,NaN,0,1.386294,0.693147,0.693147,-1.333333,0.666667,0.000000,2.772589
38197678,6999248,11,1622,1,2869,192749,30532,NaN,test,81179712,NaN,0,0.000000,1.386294,1.098612,0.000000,-1.011776,0.209962,2.484907
38197679,6999249,11,1636,1,4401,286071,35107,NaN,test,81179713,NaN,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38197680,6999250,11,1625,1,1259,978760,1232,NaN,test,81179714,NaN,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


###  Feature 4:  Mean of pairs cliente-producto

In [14]:
tic()
prod_mean_dict = prod_mean.to_dict()
client_mean_dict = client_mean.to_dict()
prod_client_mean_dict = prod_client_mean.to_dict()
tac()

Time passed: 0hour:0min:26sec


In [ ]:
def gen_pairs_mean_feature(key):
    key = tuple(key)
    product = key[0]
    client = key[1]
    
    val = prod_client_mean_dict['log_target'][(product,client)]
    if np.isnan(val):
        val = prod_mean_dict['log_target'][(product)]
        if np.isnan(val):
            val = client_mean_dict['log_target'][(client)]
            if np.isnan(val):
                val = global_mean
            
    return val

In [ ]:
tic()
data['pairs_mean'] = data[['Producto_ID', 'Cliente_ID']].apply(lambda x:gen_pairs_mean_feature(x), axis=1)
tac()

In [ ]:
data.head()

### Feature 5: Create a broad category of Brand of item (brand hypothesis)
Let's preprocess products a little bit. I borrowed some of the preprocessing from here: https://www.kaggle.com/vykhand/grupo-bimbo-inventory-demand/exploring-products

In [69]:
products  =  pd.read_csv("input-data/producto_tabla.csv")
products  =  pd.read_csv("input-data/producto_tabla.csv")
#products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)#python 2.7
products['short_name'] = products.NombreProducto.str.extract('^(\D*)')#python 3.0
#products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$')
#w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)')
products['weight'] = w[0].astype('float')*w[1].map({'Kg':1000, 'g':1})
#products['pieces'] =  products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')
products['pieces'] =  products.NombreProducto.str.extract('(\d+)p ').astype('float')
products.head()

,Producto_ID,NombreProducto,short_name,brand,weight,pieces
0,0,NO IDENTIFICADO 0,NO IDENTIFICADO,IDENTIFICADO,NaN,NaN
1,9,Capuccino Moka 750g NES 9,Capuccino Moka,NES,750,NaN
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41,Bimbollos Ext sAjonjoli,BIM,480,6
3,53,Burritos Sincro 170g CU LON 53,Burritos Sincro,LON,170,NaN
4,72,Div Tira Mini Doradita 4p 45g TR 72,Div Tira Mini Doradita,TR,45,4


In [70]:
products.tail()

,Producto_ID,NombreProducto,short_name,brand,weight,pieces
2587,49992,Tostado Integral 180g MTA WON 49992,Tostado Integral,WON,180,NaN
2588,49993,Tostado Integral 180g TAB WON 49993,Tostado Integral,WON,180,NaN
2589,49994,Tostado Int 0pct Grasa Azuc 200g WON 49994,Tostado Int,WON,200,NaN
2590,49996,Tostado Int 0pct Grasa Azuc 200g MTA WON 49996,Tostado Int,WON,200,NaN
2591,49997,Tostado Int 0pct Grasa Azuc 200g TAB WON 49997,Tostado Int,WON,200,NaN


In [71]:
products.brand.value_counts()

BIM             679
MLA             657
TR              257
LAR             182
GBI             130
WON             117
DH               95
LON              83
SAN              66
MR               64
ORO              44
CC               33
SL               32
BAR              31
SUA              20
RIC              20
MP               10
SUN               9
JMX               8
SKD               7
COR               5
MCM               5
THO               4
NAI               4
NES               3
TRI               3
MSK               2
CHK               2
PUL               2
BRL               2
KOD               2
AM                1
IDENTIFICADO      1
BRE               1
AV                1
DIF               1
CAR               1
EMB               1
MTB               1
VR                1
GV                1
NEC               1
VER               1
LC                1
Name: brand, dtype: int64

In [72]:
products.brand.nunique()

44

In [73]:
products_id_brand  = products[['Producto_ID', 'brand']].copy()

In [74]:
data = pd.merge(data, products_id_brand, on='Producto_ID')

In [75]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,log_target,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,rate_change2,rate_change3,rate_change4,Lags_sum,brand
0,NaN,7,1110,7,3301,15766,1216,2,train,42982034,1.098612,0.693147,1.386294,1.098612,1.609438,-0.541672,0.224814,-0.399194,4.787492,BIM
1,NaN,7,1110,7,3301,988589,1216,1,train,42982209,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,BIM
2,NaN,7,1110,7,3301,1255667,1216,4,train,42982269,1.609438,0.000000,0.000000,1.386294,1.386294,0.000000,-0.925513,0.000000,2.772589,BIM
3,NaN,7,1110,7,3301,1307034,1216,1,train,42982289,0.693147,0.000000,0.000000,1.098612,0.693147,0.000000,-1.382748,0.510331,1.791759,BIM
4,NaN,7,1110,7,3301,1603500,1216,1,train,42982338,0.693147,0.693147,1.386294,1.098612,0.000000,-0.759306,0.315140,1.203471,3.178054,BIM


### Feature 6: Create clusters of Products (utility hypothesis) - ramdonly pick 30 clusters

In [76]:
#Read files:
product_clusters = pd.read_csv('input-data/producto_clusters.csv')

In [77]:
product_clusters.tail()

,Producto_ID,NombreProducto,product_shortname,cluster
2586,49992,Tostado Integral 180g MTA WON 49992,Tostado Integral 180g,4
2587,49993,Tostado Integral 180g TAB WON 49993,Tostado Integral 180g,4
2588,49994,Tostado Int 0pct Grasa Azuc 200g WON 49994,Tostado Int 0pct Grasa Azuc 200g,4
2589,49996,Tostado Int 0pct Grasa Azuc 200g MTA WON 49996,Tostado Int 0pct Grasa Azuc 200g,4
2590,49997,Tostado Int 0pct Grasa Azuc 200g TAB WON 49997,Tostado Int 0pct Grasa Azuc 200g,4


In [78]:
print (product_clusters["cluster"].value_counts())

1     204
14    137
10    136
11    124
4     118
13    109
23    103
24    101
19     99
17     93
16     89
8      88
25     85
30     81
22     78
20     75
15     73
9      73
27     71
5      70
2      68
6      65
3      63
26     62
7      61
28     60
12     60
18     59
29     53
21     33
Name: cluster, dtype: int64


In [79]:
products_id_clusters = product_clusters[['Producto_ID', 'cluster']].copy()

In [80]:
products_id_clusters.tail()

,Producto_ID,cluster
2586,49992,4
2587,49993,4
2588,49994,4
2589,49996,4
2590,49997,4


In [81]:
data = pd.merge(data, products_id_clusters, on='Producto_ID')

In [82]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,...,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,rate_change2,rate_change3,rate_change4,Lags_sum,brand,cluster
0,NaN,7,1110,7,3301,15766,1216,2,train,42982034,...,0.693147,1.386294,1.098612,1.609438,-0.541672,0.224814,-0.399194,4.787492,BIM,2
1,NaN,7,1110,7,3301,988589,1216,1,train,42982209,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,BIM,2
2,NaN,7,1110,7,3301,1255667,1216,4,train,42982269,...,0.000000,0.000000,1.386294,1.386294,0.000000,-0.925513,0.000000,2.772589,BIM,2
3,NaN,7,1110,7,3301,1307034,1216,1,train,42982289,...,0.000000,0.000000,1.098612,0.693147,0.000000,-1.382748,0.510331,1.791759,BIM,2
4,NaN,7,1110,7,3301,1603500,1216,1,train,42982338,...,0.693147,1.386294,1.098612,0.000000,-0.759306,0.315140,1.203471,3.178054,BIM,2


### Feature 7: Create a category of Size of store based on Number of Agencies and Routes and Sales Channels that serve the store

In [83]:
#Determine pivot table
Rutas_per_store = data.pivot_table(values=["Ruta_SAK"], index=["Cliente_ID"], aggfunc=pd.Series.nunique)

In [84]:
Rutas_per_store.describe()

,Ruta_SAK
count,858290.000000
mean,2.426705
std,1.411257
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,47.000000


In [85]:
Agencies_per_store = data.pivot_table(values=["Agencia_ID"], index=["Cliente_ID"], aggfunc=pd.Series.nunique)

In [86]:
Agencies_per_store.describe()

,Agencia_ID
count,858290.000000
mean,1.049571
std,0.244343
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,63.000000


In [87]:
Canals_per_store = data.pivot_table(values=["Canal_ID"], index=["Cliente_ID"], aggfunc=pd.Series.nunique)

In [88]:
Canals_per_store.describe()

,Canal_ID
count,858290.000000
mean,1.006472
std,0.080898
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,4.000000


It doesn't look that we can Bin on Canal_ID or Agencia_ID since they are not at least semi evenly ditributed, but it does look like Ruta_SAK is a good option based on the percentiles distribution"

In [89]:
Rutas_per_store.rename(columns={'Ruta_SAK': 'Qty_Ruta_SAK'}, inplace=True)

In [90]:
#Mergin Routa_Sak's per client to data df
data = pd.merge(data,Rutas_per_store,right_index=True, left_on='Cliente_ID')

In [91]:
data.tail()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,...,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,rate_change2,rate_change3,rate_change4,Lags_sum,brand,cluster,Qty_Ruta_SAK
38194866,5500823,11,1338,1,5002,4741815,37202,NaN,test,79681287,...,0,0,0,0,0,0,0,TR,24,1
38196446,2406248,11,1970,1,1226,9589061,46131,NaN,test,76586712,...,0,0,0,0,0,0,0,BIM,19,1
38196921,6277674,10,1970,1,1226,9589061,46131,NaN,test,80458138,...,0,0,0,0,0,0,0,BIM,19,1
38197081,648237,11,2092,11,3903,4765705,36524,NaN,test,74828701,...,0,0,0,0,0,0,0,TR,8,1
38197649,1894706,11,3223,8,3404,4099961,48298,NaN,test,76075170,...,0,0,0,0,0,0,0,GBI,24,1


In [92]:
#Binning:
def binning(col, cut_points, labels=None):
  #Define min and max values:
  minval = col.min()
  maxval = col.max()

  #create list by adding min and max to cut_points
  break_points = [minval] + cut_points + [maxval]

  #if no labels provided, use default labels 0 ... (n-1)
  if not labels:
    labels = range(len(cut_points)+1)

  #Binning using cut function of pandas
  colBin = pd.cut(col,bins=break_points,labels=labels,include_lowest=True)
  return colBin

#Binning Qty_Ruta_SAK:
cut_points = [2,4,10]
labels = ["low","medium","high","very high"]
data["Qty_Ruta_SAK_Bin"] = binning(data["Qty_Ruta_SAK"], cut_points, labels)
print (pd.value_counts(data["Qty_Ruta_SAK_Bin"], sort=False))

low          12537899
medium       14026740
high         11559475
very high       73567
dtype: int64


In [93]:
#We don't need Qty_Ruta_Sak anymore
data.drop(['Qty_Ruta_SAK'],axis=1,inplace=True)

In [94]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,...,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,rate_change2,rate_change3,rate_change4,Lags_sum,brand,cluster,Qty_Ruta_SAK_Bin
0,NaN,7,1110,7,3301,15766,1216,2,train,42982034,...,1.386294,1.098612,1.609438,-0.541672,0.224814,-0.399194,4.787492,BIM,2,low
142133,NaN,8,1110,7,3301,15766,1216,5,train,53364884,...,0.693147,1.386294,1.098612,0.316858,-0.541672,0.224814,4.276666,BIM,2,low
418305,924190,11,1110,7,3301,15766,1216,NaN,test,75104654,...,0.000000,1.791759,1.098612,0.000000,-1.400202,0.541672,2.890372,BIM,2,low
490311,NaN,7,1110,7,3301,15766,1238,2,train,42982035,...,0.693147,1.386294,1.609438,0.334524,-0.571872,-0.184102,4.787492,BIM,2,low
655054,NaN,8,1110,7,3301,15766,1238,3,train,53364886,...,1.098612,0.693147,1.386294,0.000000,0.334524,-0.571872,4.276666,BIM,2,low


### Feature 8: Create a category of location based on zip code (embedded on town table)

In [95]:
import re 
import os
import time
towns = pd.read_csv("input-data/town_state.csv")
L = lambda x: list(map(int, re.findall('\d+', x)))[0]
towns['ZipCode'] = towns.Town.apply(L) 
towns['ZipCode'] = np.uint16(towns['ZipCode'])

In [96]:
zipcodes_df = towns[['Agencia_ID', 'ZipCode']].copy()

In [97]:
zipcodes_df.head()

,Agencia_ID,ZipCode
0,1110,2008
1,1111,2002
2,1112,2004
3,1113,2008
4,1114,2029


In [98]:
data = pd.merge(data, zipcodes_df, on='Agencia_ID')

In [99]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,...,Log_Target_mean_lag3,Log_Target_mean_lag4,rate_change2,rate_change3,rate_change4,Lags_sum,brand,cluster,Qty_Ruta_SAK_Bin,ZipCode
0,NaN,7,1110,7,3301,15766,1216,2,train,42982034,...,1.098612,1.609438,-0.541672,0.224814,-0.399194,4.787492,BIM,2,low,2008
1,NaN,8,1110,7,3301,15766,1216,5,train,53364884,...,1.386294,1.098612,0.316858,-0.541672,0.224814,4.276666,BIM,2,low,2008
2,924190,11,1110,7,3301,15766,1216,NaN,test,75104654,...,1.791759,1.098612,0.000000,-1.400202,0.541672,2.890372,BIM,2,low,2008
3,NaN,7,1110,7,3301,15766,1238,2,train,42982035,...,1.386294,1.609438,0.334524,-0.571872,-0.184102,4.787492,BIM,2,low,2008
4,NaN,8,1110,7,3301,15766,1238,3,train,53364886,...,0.693147,1.386294,0.000000,0.334524,-0.571872,4.276666,BIM,2,low,2008


In [100]:
data.apply(lambda x: len(x.unique()))

id                       6999252
Semana                         5
Agencia_ID                   552
Canal_ID                       9
Ruta_SAK                    3240
Cliente_ID                858290
Producto_ID                 1727
Demanda_uni_equil           1481
source                         3
ix                      38197681
log_target                  1481
Log_Target_mean_lag1        4296
Log_Target_mean_lag2        4848
Log_Target_mean_lag3        5369
Log_Target_mean_lag4        5342
rate_change2             2989662
rate_change3             3176242
rate_change4             3115348
Lags_sum                  386230
brand                         31
cluster                       30
Qty_Ruta_SAK_Bin               4
ZipCode                      254
dtype: int64

### Feature 9: Week of the month counter

The idea is to have an indicator of what week of the month the current data belongs. This is to see if there is a monthly pattern that the algorithm can pick up.

In [101]:
data['week_ct'] = data['Semana'].apply(lambda x: x%4)

In [102]:
data.tail()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,...,Log_Target_mean_lag4,rate_change2,rate_change3,rate_change4,Lags_sum,brand,cluster,Qty_Ruta_SAK_Bin,ZipCode,week_ct
38197676,NaN,8,1160,8,3601,827594,33198,3072,train,54683002,...,6.867974,-0.098689,-0.076554,0.183690,29.756138,BIM,17,low,2089,0
38197677,NaN,9,1160,8,3601,827594,33198,NaN,val,65086912,...,8.270013,0.143851,-0.098689,-0.076554,30.918574,BIM,17,low,2089,1
38197678,NaN,7,1160,8,3602,932176,2438,400,train,44255420,...,0.000000,-0.080673,1.092497,0.000000,13.307350,BIM,9,low,2089,3
38197679,NaN,8,1160,8,3602,932176,2438,400,train,54683003,...,0.000000,-0.063986,-0.080673,1.092497,19.301311,BIM,9,low,2089,0
38197680,NaN,9,1160,8,3602,932176,2438,NaN,val,65086913,...,6.908755,0.000000,-0.063986,-0.080673,25.295273,BIM,9,low,2089,1


### Feature 10: Clusters

#### 10.1 Client type clusters
Thanks to AbderRahman Sobh - https://www.kaggle.com/abbysobh/grupo-bimbo-inventory-demand/classifying-client-type-using-client-names/comments
for doing the great code

In [103]:
client_types = pd.read_csv('./input-data/client_types.csv',header=0)

In [104]:
client_types.head()

,Cliente_ID,NombreCliente
0,0,Individual
1,1,Oxxo Store
2,2,Individual
3,3,Small Franchise
4,4,Small Franchise


In [105]:
data = data.merge(client_types.drop_duplicates(subset=['Cliente_ID']), how="left")

In [106]:
data.tail()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,...,rate_change2,rate_change3,rate_change4,Lags_sum,brand,cluster,Qty_Ruta_SAK_Bin,ZipCode,week_ct,NombreCliente
38197676,NaN,8,1160,8,3601,827594,33198,3072,train,54683002,...,-0.098689,-0.076554,0.183690,29.756138,BIM,17,low,2089,0,Individual
38197677,NaN,9,1160,8,3601,827594,33198,NaN,val,65086912,...,0.143851,-0.098689,-0.076554,30.918574,BIM,17,low,2089,1,Individual
38197678,NaN,7,1160,8,3602,932176,2438,400,train,44255420,...,-0.080673,1.092497,0.000000,13.307350,BIM,9,low,2089,3,Small Franchise
38197679,NaN,8,1160,8,3602,932176,2438,400,train,54683003,...,-0.063986,-0.080673,1.092497,19.301311,BIM,9,low,2089,0,Small Franchise
38197680,NaN,9,1160,8,3602,932176,2438,NaN,val,65086913,...,0.000000,-0.063986,-0.080673,25.295273,BIM,9,low,2089,1,Small Franchise


#### 10.2 Client type clusters
We are going to load a previous calculate tables of Agencia,Ruta, Producto and Cliente Clusters based on demand mean and std

In [ ]:
import h2o
import imp
from h2o.estimators.kmeans import H2OKMeansEstimator

In [ ]:
# Start a local instance of the H2O engine.
h2o.init();

In [ ]:
from sklearn.cluster import KMeans
import math as math
from matplotlib import pyplot as plt
%matplotlib inline

#### Let's try to find the right amount of clusters

There are three diagnostics that will be used to help with determining the number of clusters: total within cluster sum of squares, AIC, and BIC.

Total within cluster sum of squares measures sums the distance from each point in a cluster to that point's assigned cluster center. This is the minimization criteria of kmeans. The standard guideline for picking the number of clusters is to look for a 'knee' in the plot, showing where the total within sum of squares stops decreasing rapidly. Total within cluster sum of squares can be difficult to intepret, with the criteria being to look for an arbitrary knee in the plot.

With this challenge from total within cluster sum of squares, we will also use two merit statistics for determining the number of clusters. AIC and BIC are both measures of the relative quality of a statistical model. AIC and BIC introduce penality terms for the number of parameters in the model to counter the problem of overfitting; BIC has a larger penality term than AIC. With these merit statistics one is to look for the number of clusters that minimize the statistic.

Here we build a method for extracting the inputs for each diagnostics and calculating the AIC and BIC values on a model. Each model is then inspected by the method and the results plotted for quick analysis.


In [ ]:
def diagnostics_from_clusteringmodel(model):
    total_within_sumofsquares = model.tot_withinss()
    number_of_clusters = len(model.centers())
    number_of_dimensions = len(model.centers()[0])
    number_of_rows = sum(model.size())
    
    aic = total_within_sumofsquares + 2 * number_of_dimensions * number_of_clusters
    bic = total_within_sumofsquares + math.log(number_of_rows) * number_of_dimensions * number_of_clusters
    
    return {'Clusters':number_of_clusters,
            'Total Within SS':total_within_sumofsquares, 
            'AIC':aic, 
            'BIC':bic}

In [ ]:
def cluster_and_plot(feat_name, k_range):

    global data
    
    # run clustering by demand using all data set and our pairs_mean feature
    feat = data.loc[:,feat_name]
    tar = data.loc[:,'pairs_mean']
    feat_tar = pd.concat([feat, tar], axis=1)
        
    # group by feature
    grouped_ft = feat_tar.groupby(feat_name)
    grouped_ft = pd.merge(grouped_ft.median().reset_index(), grouped_ft.std().reset_index(), how='left', on=[feat_name])
    
    demand_info_unique = grouped_ft.iloc[:,1:]
    demand_info_unique[pd.isnull(demand_info_unique)] = 0
        
    # we use the kmeans clustering algorithm
    h2odf = h2o.H2OFrame(demand_info_unique)
    results = [H2OKMeansEstimator(k=clusters, init="Random", seed=2, standardize=True) for clusters in k_range]
    for estimator in results:
        estimator.train(x = h2odf.col_names, training_frame = h2odf)

    diagnostics = pd.DataFrame( [diagnostics_from_clusteringmodel(model) for model in results])
    diagnostics.set_index('Clusters', inplace=True)
    diagnostics.plot(kind='line');


In [ ]:
agencia_k_range = range(10, 50, 5)
tic()
cluster_and_plot('Agencia_ID', agencia_k_range)
tac()

It Looks Like Clusters = 20 is a good choice

In [ ]:
ruta_k_range = range(60, 300, 30)
tic()
cluster_and_plot('Ruta_SAK', ruta_k_range)
tac()

It Looks Like Clusters = 80 is a good choice

In [ ]:
producto_k_range = range(30, 150, 15)
tic()
cluster_and_plot('Producto_ID', producto_k_range)
tac()

It Looks Like Clusters = 45 is a good choice

In [ ]:
tic()
cliente_k_range = [10,100,300,500,1000]
cluster_and_plot('Cliente_ID', cliente_k_range)
tac()

It looks like 300 or 400 would work

#### Now we the right amount of centroids, we process the clustering and fitting

In [ ]:
# --- HYPERPARAMETERS FOR FEATURE EXTRACTION ---

# number of clusters to group depot/route/produc (if 0 will not be added as feature)
num_clusters_agencia = 20
num_clusters_ruta = 80
num_clusters_producto = 45
num_clusters_cliente = 400

In [ ]:
def cluster_and_save(feat_name, num_clusters):
    '''
    Input: 
        - idx: the index of the feature we want to cluster, one-hot-encode, and add to our features
        - num_clusters: the number of clusters we want to use to group the feature values
    '''   
    global data

    # run clustering by demand using info from week 3-9
    feat = data.loc[:, feat_name]
    tar = data.loc[:, 'pairs_mean']
    feat_tar = pd.concat([feat, tar], axis=1)
        
    # group by feature
    grouped_ft = feat_tar.groupby(feat_name)
    grouped_ft = pd.merge(grouped_ft.median().reset_index(), grouped_ft.std().reset_index(), how='left', on=[feat_name])
    
    demand_info_unique = grouped_ft.iloc[:,1:]
    demand_info_unique[pd.isnull(demand_info_unique)] = 0
    
    # we use the kmeans clustering algorithm
    h2odf = h2o.H2OFrame(demand_info_unique)
    kmeans = H2OKMeansEstimator(k=num_clusters, init="Random", seed=2, standardize=True)

    kmeans.train(x = h2odf.col_names, training_frame = h2odf)
    clusters_h2odf = kmeans.predict(h2odf)
    clustersdf = clusters_h2odf.as_data_frame(True)
    clusters = clustersdf.predict.get_values()
    
    # plot demand/cluster
    plt.figure(figsize=(15, 3))
    for c in range(num_clusters):
        d_median = demand_info_unique.iloc[clusters==c,0]
        d_std = demand_info_unique.iloc[clusters==c,1]
        plt.plot(d_median,d_std,'.')
        plt.xlabel('median')
        plt.ylabel('std')
    plt.savefig(path_datadrive+'h2o-clustByDem_{}_{}'.format(feat_name, num_clusters))
    
    # create new dataframe to save the mapping from feature ID to cluster ID
    feat_clust_map = pd.DataFrame(data = grouped_ft.iloc[:,0], columns=[feat_name])
    feat_clust_map.insert(1, feat_name+'_clust_ID', clusters)

    # save the new feature in files for others to use 
    feat_clust_map.to_csv(path_datadrive+"h2o-clustByDem_{}_{}.csv".format(feat_name, num_clusters), index=False)

In [ ]:
tic()
print("Agencia Clusters")
cluster_and_save('Agencia_ID', num_clusters_agencia)
tac()

In [ ]:
tic()
print("Ruta Clusters")
cluster_and_save('Ruta_SAK', num_clusters_ruta)
tac()

In [ ]:
tic()
print("Producto Clusters")
cluster_and_save('Producto_ID', num_clusters_producto)
tac()

In [ ]:
tic()
print("Cliente Clusters")
cluster_and_save('Cliente_ID', num_clusters_cliente)
tac()

In [ ]:
#Read files:
agencia_by_dem_clust = pd.read_csv('input-data/h2o-clustByDem_Agencia_ID_'+ num_clusters_agencia +'.csv')
ruta_by_dem_clust = pd.read_csv('input-data/h2o-clustByDem_Ruta_SAK_'+ num_clusters_ruta +'.csv')
prod_by_dem_clust = pd.read_csv('input-data/h2o-clustByDem_Producto_ID_'+ num_clusters_producto +'.csv')
cliente_by_dem_clust = pd.read_csv('input-data/h2o-clustByDem_Cliente_ID_'+ num_clusters_cliente +'.csv')

In [ ]:
data = pd.merge(data, prod_by_dem_clust, on='Producto_ID')
data = pd.merge(data, ruta_by_dem_clust, on='Ruta_SAK')
data = pd.merge(data, agencia_by_dem_clust, on='Agencia_ID')
data = pd.merge(data, cliente_by_dem_clust, on='Cliente_ID')

In [ ]:
data.tail()

## Numerical and One-Hot Coding of Categorical variables
Since scikit-learn accepts only numerical variables, so i have to convert all categories of nominal variables into numeric types.

Lets start with coding all low cardinality object/nominal categorical variables (brand, Qty_Ruta_SAK_Bin, NombreCliente)  as numeric using ‘LabelEncoder’ from sklearn’s preprocessing module.

In [119]:
print (data.dtypes)

id                      float64
Semana                     int8
Agencia_ID               uint16
Canal_ID                   int8
Ruta_SAK                  int32
Cliente_ID                int32
Producto_ID               int32
Demanda_uni_equil       float64
source                   object
ix                        int64
log_target              float64
Log_Target_mean_lag1    float64
Log_Target_mean_lag2    float64
Log_Target_mean_lag3    float64
Log_Target_mean_lag4    float64
rate_change2            float64
rate_change3            float64
rate_change4            float64
Lags_sum                float64
brand                    object
cluster                   int64
Qty_Ruta_SAK_Bin         object
ZipCode                  uint16
week_ct                   int64
NombreCliente            object
Producto_ID_clust_ID      int64
Ruta_SAK_clust_ID         int64
Agencia_ID_clust_ID       int64
dtype: object


In [120]:
#Import library:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

var_mod = ['brand', 'Qty_Ruta_SAK_Bin', 'NombreCliente']
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [121]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,source,ix,...,Lags_sum,brand,cluster,Qty_Ruta_SAK_Bin,ZipCode,week_ct,NombreCliente,Producto_ID_clust_ID,Ruta_SAK_clust_ID,Agencia_ID_clust_ID
0,NaN,7,1110,7,3301,15766,1216,2,train,42982034,...,4.787492,1,2,1,2008,3,11,61,99,9
1,NaN,8,1110,7,3301,15766,1216,5,train,53364884,...,4.276666,1,2,1,2008,0,11,61,99,9
2,924190,11,1110,7,3301,15766,1216,NaN,test,75104654,...,2.890372,1,2,1,2008,3,11,61,99,9
3,NaN,7,1110,7,3301,988589,1216,1,train,42982209,...,0.000000,1,2,1,2008,3,8,61,99,9
4,NaN,7,1110,7,3301,1255667,1216,4,train,42982269,...,2.772589,1,2,1,2008,3,8,61,99,9


One-Hot-Coding refers to creating dummy variables, one for each category of a categorical variable. For example, the 'cluster' variable has 30 categories. One hot coding will remove this variable and generate 30 new variables. Each will have binary numbers – 0 (if the category is not present) and 1(if category is present).
Categorical variables are intentionally (for censorship) or implicitly encoded as numerical variables in order to be used as features in any given model.

e.g. [house, car, tooth, car] becomes [0,1,2,1].

This imparts an ordinal property to the variable, i.e. house < car < tooth.

As this is ordinal characteristic is usually not desired, one hot encoding is necessary for the proper representation of the distinct elements of the variable.

-- This can be done using ‘get_dummies’ function of Pandas.


In [122]:
#One Hot Coding: you need python 3 and 128GB ram for this
#tic()
#data = pd.get_dummies(data, columns=['Canal_ID','brand','cluster','Qty_Ruta_SAK_Bin'])
#tac()

I decided NOT to one-hot-encode, since the data would grew to unmanageable size

In [123]:
data.dtypes

id                      float64
Semana                     int8
Agencia_ID               uint16
Canal_ID                   int8
Ruta_SAK                  int32
Cliente_ID                int32
Producto_ID               int32
Demanda_uni_equil       float64
source                   object
ix                        int64
log_target              float64
Log_Target_mean_lag1    float64
Log_Target_mean_lag2    float64
Log_Target_mean_lag3    float64
Log_Target_mean_lag4    float64
rate_change2            float64
rate_change3            float64
rate_change4            float64
Lags_sum                float64
brand                     int64
cluster                   int64
Qty_Ruta_SAK_Bin          int64
ZipCode                  uint16
week_ct                   int64
NombreCliente             int64
Producto_ID_clust_ID      int64
Ruta_SAK_clust_ID         int64
Agencia_ID_clust_ID       int64
dtype: object

## 5\. Exporting Data

In [124]:
#Divide into test and train:
import csv

tic()
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]
if (use_validation):
    val = data.loc[data['source']=="val"]
    val.drop('Demanda_uni_equil',axis=1,inplace=True) # here all Demanda_uni_equil = nan
    # we merge back the val_targets to the val dataset
    val = pd.merge(val,val_target, on='ix') # we add the true Demanda_uni_equil back
    val['log_target'] = np.log1p(dval["Demanda_uni_equil"]) # adds the log_target col

#Drop unnecessary columns: note - we are dropping Demanda_uni_equil since we replaced it by log_target
dtrain.drop(['source','id','Demanda_uni_equil','ix'],axis=1,inplace=True)
dval.drop(['source','id','Demanda_uni_equil','ix'],axis=1,inplace=True)
dtest.drop(['source','Demanda_uni_equil','ix'],axis=1,inplace=True)

#Export files as modified versions:
if (use_validation):
    train.to_csv("./input-data/train_modified_holdout.csv", index=False, quoting=csv.QUOTE_NONE)
    val.to_csv("./input-data/val_modified_holdout.csv", index=False, quoting=csv.QUOTE_NONE)
    test.to_csv("./input-data/test_modified_holdout.csv", index=False, quoting=csv.QUOTE_NONE)
else:
    train.to_csv("./input-data/train_modified.csv", index=False, quoting=csv.QUOTE_NONE)
    test.to_csv("./input-data/test_modified.csv", index=False, quoting=csv.QUOTE_NONE)  
tac()


C:\Anaconda\envs\py35\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\envs\py35\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Time passed: 0hour:12min:24sec


C:\Anaconda\envs\py35\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [130]:
dtrain.tail()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,log_target,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,...,Lags_sum,brand,cluster,Qty_Ruta_SAK_Bin,ZipCode,week_ct,NombreCliente,Producto_ID_clust_ID,Ruta_SAK_clust_ID,Agencia_ID_clust_ID
38197656,8,1156,4,1855,4719798,37086,2.397895,2.397895,0,0.000000,...,2.397895,11,29,1,2018,0,9,17,25,15
38197658,8,1156,4,1855,1276758,35989,1.386294,0.000000,0,1.098612,...,1.098612,11,21,1,2018,0,10,61,25,15
38197660,7,1156,4,1855,4385114,35989,0.693147,1.098612,0,0.000000,...,1.098612,11,21,1,2018,3,9,61,25,15
38197672,8,1156,4,1855,4533291,35989,0.693147,0.000000,0,0.000000,...,0.000000,11,21,1,2018,0,9,61,25,15
38197673,8,1156,4,1855,4216198,35989,1.098612,0.000000,0,0.000000,...,0.000000,11,21,1,2018,0,10,61,25,15
